In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Prune, Train Attack Pipeline

In [9]:
pgd_success_rates = []
cw_success_rates = []
all_accuracies = []
for j in tqdm(range(10)):
    accuracies = []
    pgd_success_rate = []
    cw_success_rate = []
    pruning_ratios = [.0, .3, .5, .7, .8, .9, .95, .97, .98]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model = tf.keras.models.load_model('./saved-models/mini-pipeline-baseline-model')
        for i in range(index + 1):
            if i != index:
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=False)
            if i == index:
                print('final pruning and eval')
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=True)
                accuracies.append(model.evaluate(x_test, y_test, verbose=0))
                pgd_success_rate.append(pgd_attack(model))
                cw_success_rate.append(cw2_attack(model))
                
    all_accuracies.append(accuracies)
    pgd_success_rates.append(pgd_success_rate)
    cw_success_rates.append(cw_success_rate)


  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]

final pruning and eval
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 5s 82us/sample - loss: 1.4984 - accuracy: 0.9644 - val_loss: 1.4965 - val_accuracy: 0.9659
Epoch 2/100
60000/60000 [==============================] - 5s 76us/sample - loss: 1.4907 - accuracy: 0.9717 - val_loss: 1.4923 - val_accuracy: 0.9688
Epoch 3/100
60000/60000 [==============================] - 4s 69us/sample - loss: 1.4865 - accuracy: 0.9758 - val_loss: 1.4898 - val_accuracy: 0.9719
Epoch 4/100
60000/60000 [==============================] - 4s 70us/sample - loss: 1.4831 - accuracy: 0.9789 - val_loss: 1.4901 - val_accuracy: 0.9712
Epoch 5/100
60000/60000 [==============================] - 5s 76us/sample - loss: 1.4806 - accuracy: 0.9811 - val_loss: 1.4894 - val_accuracy: 0.9728
Epoch 6/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4791 - accuracy: 0.9825 - val_loss: 1.4853 - val_accuracy: 0.9762
Epoch 7/100
60000/60000 [==


1it [11:28, 688.44s/it]

self.p 2
flatten(p) TensorFlowTensor(<tf.Tensor: shape=(500, 784), dtype=float32, numpy=
                 array([[1.1920929e-07, 1.7522573e-03, 9.1007352e-04, ..., 0.0000000e+00,
                         5.9604645e-08, 2.9802322e-08],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        ...,
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00]], dtype

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 101us/sample - loss: 1.4983 - accuracy: 0.9646 - val_loss: 1.5001 - val_accuracy: 0.9634
Epoch 2/2
60000/60000 [==============================] - 5s 80us/sample - loss: 1.4906 - accuracy: 0.9718 - val_loss: 1.4946 - val_accuracy: 0.9674
final pruning and eval
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 5s 89us/sample - loss: 1.4835 - accuracy: 0.9787 - val_loss: 1.4896 - val_accuracy: 0.9721
Epoch 2/100
60000/60000 [==============================] - 6s 96us/sample - loss: 1.4804 - accuracy: 0.9816 - val_loss: 1.4885 - val_accuracy: 0.9731
Epoch 3/100
60000/60000 [==============================] - 5s 78us/sample - loss: 1.4792 - accuracy: 0.9828 - val_loss: 1.4873 - val_accuracy: 0.9745
Epoch 4/100
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4772 - accuracy: 0.9845 - val_loss: 1.4859 - v

...


9it [1:56:00, 773.37s/it]
 20%|██        | 2/10 [4:01:01<16:18:31, 7338.88s/it]
0it [00:00, ?it/s]

self.p 2
flatten(p) TensorFlowTensor(<tf.Tensor: shape=(500, 784), dtype=float32, numpy=
                 array([[4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        ...,
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07]], dtype

final pruning and eval
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4983 - accuracy: 0.9644 - val_loss: 1.4973 - val_accuracy: 0.9648
Epoch 2/100
60000/60000 [==============================] - 4s 60us/sample - loss: 1.4908 - accuracy: 0.9716 - val_loss: 1.4922 - val_accuracy: 0.9694
Epoch 3/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4858 - accuracy: 0.9762 - val_loss: 1.4908 - val_accuracy: 0.9709
Epoch 4/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4839 - accuracy: 0.9781 - val_loss: 1.4896 - val_accuracy: 0.9720
Epoch 5/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4810 - accuracy: 0.9811 - val_loss: 1.4873 - val_accuracy: 0.9742
Epoch 6/100
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4797 - accuracy: 0.9816 - val_loss: 1.4864 - val_accuracy: 0.9756
Epoch 7/100
60000/60000 [==

0it [06:15, ?it/s]
 20%|██        | 2/10 [4:07:16<16:29:06, 7418.25s/it]


KeyboardInterrupt: 

In [4]:
def get_average_success_rates(all_sucess_rates):
    success_per_pruning_rate=[]
    for i in range(len(all_success_rates)):
        for j in range(len(all_success_rates[i])):

            try:
                success_per_pruning_rate[j].append(all_success_rates[i][j])
            except:
                success_per_pruning_rate.append([])
                success_per_pruning_rate[j].append(all_success_rates[i][j])
    avg_success_per_pruning_rate = [sum(x)/len(x) for x in success_per_pruning_rate];avg_success_per_pruning_rate
    return avg_success_per_pruning_rate

In [3]:
def get_average_accuracies(all_accuracies):
    acc_per_pruning_rate=[]
    for i in range(len(all_accuracies)):
        for j in range(len(all_accuracies[i])):

            try:
                acc_per_pruning_rate[j].append(all_accuracies[i][j][1])
            except:
                acc_per_pruning_rate.append([])
                acc_per_pruning_rate[j].append(all_accuracies[i][j][1])
    avg_acc_per_pruning_rate = [sum(x)/len(x) for x in acc_per_pruning_rate]; avg_acc_per_pruning_rate
    return avg_acc_per_pruning_rate

# Helper Functions

In [1]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model

def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 3) or (index == 6):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            flat_weights_to_prune_df = pd.DataFrame(flat_weights_to_prune)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+1] = weights_reshaped
    #print(weights_to_prune)
    return weights_to_prune

def pgd_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.LinfProjectedGradientDescentAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/500

def cw2_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.L2CarliniWagnerAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[.5]
    )
    return np.count_nonzero(adversarials[2])/len(y)

# Load Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:500].reshape(500,28*28))
y = tf.convert_to_tensor([y_train[:500]])[0];

# Define Model

In [ ]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False,
                                       name='pruned_weights')
        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        self.pruned_w = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        
        x = self.dense1(inputs)
        #print(x.shape)
        x = self.dense2(x)
        #print(x.shape)
        return self.dense3(x)
    


# Compile and Train Model

In [ ]:
model = LeNet300_100()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
              
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=64,
          epochs=2,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )
model.save('./saved-models/mini-pipeline-mlp-baseline-model')